# I - BiciMAD: Análisis descriptivo fuentes de datos

<br/>
<div class="alert alert-block alert-success">El objetivo de este análisis es descubrir y entender las diversas fuentes de datos en las que se basa el Trabajo de Fin de Máster.  </div>

Las fuentes de datos son open source y se encuetran disponibles en las siguientes webs:

<a href="https://datos.madrid.es/portal/site/egob/menuitem.9e1e2f6404558187cf35cf3584f1a5a0/?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default">Portal de datos abiertos del ayuntamiento de madrid</a>

<a href="http://opendata.emtmadrid.es/">Portal de datos abiertos "Open Data" de la EMT (Empresa Municipal de Transportes de Madrid)</a>
 
<br/>
Este notebook se ejecuta en Python 3.5

<table style="border: none">
   <tr style="border: none">
       <td style="border: none"><img src="https://ecomovilidad.net/wp-content/uploads/2015/03/BiciMad-t%C3%B3tem-630x340.jpg" alt="Icon"></td>
   </tr>
</table>

El alquiler de bicicleta pública es un servicio dirigido a todos los ciudadanos y visitantes de la ciudad de Madrid, como elemento alternativo de transporte limpio que contribuye a un modelo de movilidad más sostenible y al fomento de hábitos de transporte más equilibrados y saludables.

## Fuentes datos

1. [Alta de usuarios y usos por día del servicio público de bicicleta eléctrica](#uso)
2. [Bases de bicicletas](#bases)


<a id="uso"></a>
## 1. Alta de usuarios y usos por día del servicio público de bicicleta eléctrica

Este conjunto de datos se presenta la información desagregada día a día de:

 - Usuarios: nuevas altas diarias en el sistema en sus dos modalidades (abonos anuales y ocasionales)
 - Usos: numero total diario de usos de la bicicleta. Esta información se presenta en csv (datos del último mes y datos acumulados desde el inicio de la prestación del servicio) y también en excel con estadísticas y gráficas asociadas. Datos desde su puesta en funcionamiento en junio de 2014.

Descarga de los datos.

Tip: usar !pip install wget para instalar la librería wget

In [1]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
# Import the boto library
import ibm_boto3
from ibm_botocore.client import Config
import os
import json
import warnings
import urllib
import time
warnings.filterwarnings('ignore')

import wget, os

# Create folder
data_dir = 'DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

links = ['https://datos.madrid.es/egob/catalogo/213155-1-bicimad-usos-usuarios.xls'
        ,'https://datos.madrid.es/egob/catalogo/208327-0-transporte-bicicletas-bicimad.xls']

# Download the links to the folder
filepath = os.path.join(data_dir, 'bicimad-usos-usuarios.xls')

if not os.path.isfile(filepath):
    print(links[0])
    urllib.request.urlretrieve(links[0], filepath)
    
    # Download the links to the folder
filepath = os.path.join(data_dir, 'bases_bicimad_data.xls')

if not os.path.isfile(filepath):
    print(links[1])
    urllib.request.urlretrieve(links[1], filepath)

# List the files in the STYLE_DATA folder        
!ls DATA

https://datos.madrid.es/egob/catalogo/213155-1-bicimad-usos-usuarios.xls
https://datos.madrid.es/egob/catalogo/208327-0-transporte-bicicletas-bicimad.xls
bases_bicimad_data.xls	bicimad-usos-usuarios.xls


In [3]:
import pandas as pd
usoDataDf = pd.read_excel("DATA/bicimad-usos-usuarios.xls")
usoDataDf.head()

,DIA,Usos bicis abono anual,Usos bicis abono ocasional,Usos bicis total,Unnamed: 4,Usos abono anual acumulado,Usos abono ocasional acumulado,Usos total acumulado desde inicio,Unnamed: 8,Acumulado mes total
0,2014-06-23,17,0,17,NaN,17,0,17,NaN,NaN
1,2014-06-24,24,0,24,NaN,41,0,41,NaN,NaN
2,2014-06-25,105,0,105,NaN,146,0,146,NaN,NaN
3,2014-06-26,120,0,120,NaN,266,0,266,NaN,NaN
4,2014-06-27,131,0,131,NaN,397,0,397,NaN,NaN


In [4]:
usoDataDf.describe()

,Usos bicis abono anual,Usos bicis abono ocasional,Usos bicis total,Unnamed: 4,Usos abono anual acumulado,Usos abono ocasional acumulado,Usos total acumulado desde inicio,Unnamed: 8,Acumulado mes total
count,1439.000000,1439.000000,1439.000000,0.0,1.439000e+03,1439.000000,1.439000e+03,0.0,48.000000
mean,7640.935372,165.849896,7806.785268,NaN,4.985771e+06,105533.335650,5.091304e+06,NaN,234040.916667
std,3113.422239,142.017212,3167.396403,NaN,3.335640e+06,75967.218624,3.411534e+06,NaN,87899.028995
min,17.000000,0.000000,17.000000,NaN,1.700000e+01,0.000000,1.700000e+01,NaN,726.000000
25%,5114.500000,55.000000,5224.000000,NaN,1.762976e+06,32746.500000,1.795723e+06,NaN,173373.750000
50%,7711.000000,141.000000,7928.000000,NaN,5.009934e+06,108655.000000,5.118589e+06,NaN,235803.500000
75%,9838.000000,230.500000,10012.500000,NaN,7.632890e+06,167608.000000,7.800498e+06,NaN,303368.750000
max,15044.000000,745.000000,15308.000000,NaN,1.099531e+07,238658.000000,1.123396e+07,NaN,384072.000000


### ¿Qué fechas contiene el dataset?

In [5]:
# data preparation
usoDataDf.columns

Index(['DIA', 'Usos bicis abono anual ', 'Usos bicis abono ocasional ',
       'Usos bicis total', 'Unnamed: 4', 'Usos abono anual acumulado',
       'Usos abono ocasional acumulado', 'Usos total acumulado desde inicio',
       'Unnamed: 8', 'Acumulado mes total'],
      dtype='object')

In [21]:
min(usoDataDf["DIA"])

Timestamp('2014-06-23 00:00:00')

In [22]:
max(usoDataDf["DIA"])

Timestamp('2018-05-31 00:00:00')

Se observan datos desde el 23 de junio de 2014 hasta el 31 de mayo de 2018

### Preparación de los datos

In [6]:
# data preparation
usoDiarioDataDf = usoDataDf.ix[:,['DIA','Usos bicis abono anual ','Usos bicis abono ocasional ','Usos bicis total']]
usoDiarioDataDf.head()

,DIA,Usos bicis abono anual,Usos bicis abono ocasional,Usos bicis total
0,2014-06-23,17,0,17
1,2014-06-24,24,0,24
2,2014-06-25,105,0,105
3,2014-06-26,120,0,120
4,2014-06-27,131,0,131


In [7]:
usoDiarioDataDf.columns = ["date", "anual", "occasional", "total"]
usoDiarioDataDf.fillna(0, inplace=True)
usoDiarioDataDf["day"] = usoDiarioDataDf["date"].apply(lambda date: date.weekday())
usoDiarioDataDf["month"] = usoDiarioDataDf["date"].apply(lambda date: date.month)

In [8]:
#plot lib
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout, Margin, Data, Scattermapbox, Marker
init_notebook_mode(connected=True)

data = [
    Scatter(
        x=usoDiarioDataDf.date, 
        y=usoDiarioDataDf.anual,
        name="Usuario bono anual",
    ),
    Scatter(
        x=usoDiarioDataDf.date, 
        y=usoDiarioDataDf.occasional,
        name="Usuario ocasional",
    ),
     
]
layout = Layout(
    yaxis=dict(title="DAU"),
    title='Actividad diaria usuarios BiciMAD',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Se oberva como el uso de los abonos anuales se incrementa desde la primavera hasta que finaliza el verano, con un descenso/valle destacado durante el mes de agosto.

In [27]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
data = [
    Bar(
        x=[days[day] for day in usoDiarioDataDf.groupby("day").agg({"anual": "mean"}).index], 
        y=round(usoDiarioDataDf.groupby("day").agg({"anual": "mean"}).anual,2),
        name="anual user",
    ),
    Bar(
        x=[days[day] for day in usoDiarioDataDf.groupby("day").agg({"occasional": "mean"}).index], 
        y=round(usoDiarioDataDf.groupby("day").agg({"occasional": "mean"}).occasional,2),
        name="occasional user",
    ),
]
layout = Layout(
    barmode='stack',
    yaxis=dict(title="users"),
    title='BiciMAD users per day of the week',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

months = ["", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
data = [
    Bar(
        x=[months[month] for month in usoDiarioDataDf.groupby("month").agg({"anual": "mean"}).index], 
        y=round(usoDiarioDataDf.groupby("month").agg({"anual": "mean"}).anual,2),
        name="anual user",
    ),
    Bar(
        x=[months[month] for month in usoDiarioDataDf.groupby("month").agg({"occasional": "mean"}).index], 
        y=round(usoDiarioDataDf.groupby("month").agg({"occasional": "mean"}).occasional,2),
        name="occasional user",
    ),
]
layout = Layout(
    barmode='stack',
    yaxis=dict(title="users"),
    title='Actividad diaria usuarios BiciMAD por mes',
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Se observa coomo el el uso ocasional aumenta el fin de semana. Una causa de este aumento estaría motivado por el turismo y el ocio. 
Y un uso más elevado entre semana por su uso como medio de transporte en horario laboral.

<a id="bases"></a>
## 2. Bases de bicicletas

- Cada estación se compone del tótem (CIC) que es el centro de interacción con el ciclista y de los anclajes para la colocación de las bicicletas.
- El servicio de bicicleta pública está sujeto a unas tarifas aprobadas por el Ayuntamiento de Madrid, las cuales incluyen un seguro que cubre cualquier percance sufrido durante su utilización. 
- El horario de funcionamiento es de forma ininterrumpida durante las 24 horas del día los 365 días del año.
- Para este descriptivo se utliza una fuente de datos estática.

In [10]:
import pandas as pd
basesDataDf = pd.read_excel("DATA/bases_bicimad_data.xls")
basesDataDf.head()

,Número,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Número de Plazas
0,1a,440394.55,4474280.53,2014-06-23,01 CENTRO,01-06 SOL,"PUERTA DEL SOL, PLAZA, DE LA",1 B,24
1,1b,440404.21,4474287.39,2014-06-23,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",1,24
2,2,440134.83,4474678.23,2014-06-23,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,27
3,3,440012.98,4475760.68,2014-06-23,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,18
4,4,440396.40,4475565.36,2014-06-23,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,27


In [11]:
basesDataDf.shape

(171, 9)

Existen 171 estaciones operativas.

In [12]:
sum(basesDataDf['Número de Plazas'])

4146

Existen un total 4.146 plazas de bicicletas entre las estaciones de madrid

In [13]:
basesDistritoDf = basesDataDf.groupby("Distrito", as_index=False)\
                                        .agg({"Número": "count", "Número de Plazas": "sum"})

In [14]:
basesDistritoDf

,Distrito,Número de Plazas,Número
0,01 CENTRO,1476,61
1,02 ARGANZUELA,288,12
2,03 RETIRO,549,22
3,04 SALAMANCA,660,27
4,05 CHAMARTÍN,312,13
5,06 TETUÁN,168,7
6,07 CHAMBERÍ,450,19
7,09 MONCLOA-ARAVACA,243,10


In [15]:
basesDistritoDf.columns = ["district", "Bases", "Spots"]
basesDistritoDf.district = basesDistritoDf.district.apply(lambda x: x[2:])
basesDistritoDf.sort_values("Bases", ascending=False, inplace=True)

In [29]:
data = [
    Bar(
        x=basesDistritoDf.district, 
        y=basesDistritoDf.Bases,
        name="Bases"
    ),
    Scatter(
        x=basesDistritoDf.district, 
        y=basesDistritoDf.Spots,
        name="Plazas",
        yaxis="y2"
    )
]
layout = Layout(
    #barmode="stack",
    yaxis=dict(title="Bases"),
    yaxis2=dict(title="Plazas", side="right", overlaying='y'),
    title='BiciMAD Bases y número de plazas por distrito',
    margin=Margin(b=150)
)
fig = Figure(data=data, layout=layout)
iplot(fig)

Observamos como la bases de las estaciones se contentran en el núcleo urbano(centro), y a medida que nos alejamos, van dismnuyendo. El número de bicis es proporncioal al número de estaciones dado que cada estación dispone de 24 anclajes de bicicletas.